In [1]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from pandas import read_html

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Constants for accessing DataFrame data pulled from web
QB = 0
RB = 1
WR = 2
TE = 3
DEF = 4
KICKER = 5

In [3]:
#Set to create unique in house DataFrame 

tupOffense = [('', 'Bye'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

In [4]:
tupDef = [('', 'Bye'),
          ('', 'Pts*'),
          ('Def', 'Sack'), 
          ('Def', 'Int'),
          ('Def', 'Saf'), 
          ('Def', 'FR'), 
          ('Def', 'Blk'), 
          ('Def', 'TD'), 
          ('Def', 'PA'),
          ('Def', 'PassYds/G'), 
          ('Def', 'RushYds/G'), 
          ('Def', 'TotYds/G')
         ]

In [5]:
tupKicker = [('', 'Bye'), 
             ('', 'Pts*'),          
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [6]:
#Set to create unique in house DataFrame of master Dataframe that will have all players and seasons
masterTup = [('', 'PPR'),
             ('', 'Position'),
             ('', 'Team'),
             ('', 'Bye'),
             ('', 'Pts'),
             ('', 'Week'),
             ('', 'Year'),
             ('', 'Game'),
             ('', 'Opp'),
             ('Passing', 'Att'),
             ('Passing', 'Cmp'),
             ('Passing', 'Yds'),
             ('Passing', 'TD'),
             ('Passing', 'Int'),
             ('Passing', '2Pt'),
             ('Rushing', 'Att'),
             ('Rushing', 'Yds'),
             ('Rushing', 'TD'),
             ('Rushing', '2Pt'),
             ('Receiving', 'Rec'),
             ('Receiving', 'Yds'),
             ('Receiving', 'TD'),
             ('Receiving', '2Pt'),
             ('Fumbles', 'FL'),
             ('Fumbles', 'TD'),
             ('Defense', 'Sack'),
             ('Defense', 'Int'),
             ('Defense', 'Saf'),
             ('Defense', 'FR'), 
             ('Defense', 'Blk'),
             ('Defense', 'TD'),
             ('Defense', 'PA'),
             ('Defense', 'PassYds/G'),
             ('Defense', 'RushYds/G'),
             ('Defense', 'TotYds/G'),
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [7]:
#Set to change the index names 

indexDEF = ['Bye', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds/G', 'RushYds/G', 'TotYds/G']

indexKicker = ['Bye', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+']


In [8]:
#Load data from web for 2017 season
def loadWebData(frameArray):
    url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
    url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
    url_wr_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
    url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr=2017&wk=all&rules=1'
    url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'
    url_kicker_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=K&yr=2017&wk=all&rules=1'
    
    #Array [0] is where the data is stored. [1] stores the rules 
    frameArray.insert(QB, pd.io.html.read_html(url_qb_season17)[0])
    frameArray.insert(RB, pd.io.html.read_html(url_rb_season17)[0])
    frameArray.insert(WR, pd.io.html.read_html(url_wr_season17)[0])
    frameArray.insert(TE, pd.io.html.read_html(url_te_season17)[0])
    frameArray.insert(DEF, pd.io.html.read_html(url_def_season17)[0]) 
    frameArray.insert(KICKER, pd.io.html.read_html(url_kicker_season17)[0]) 

In [9]:
#Fix name that contains the next line string '\xa0'
#nameIndex: Empty index that will contain the fixed name
#data: Pandas Series type that contains the names of each player
def nameFix(nameIndex, data):
    #Get name from Player column and save to nameIndex
    NAME_FIX_OFFSET = 3
    index = nameIndex

    for i in data:
        index.append(i)


    size = len(index)

    for i in range(size):
        a1 = index[i]
        count = 0

        for x in a1:
            count += 1
            if x == u'\xa0':
                break

        count -= NAME_FIX_OFFSET
        index[i] = a1[:count]
    
    return index

In [10]:
def renameIndex(dframe, teamName):

    d1 = dict()
    index = 0

    for i in teamName:
        d1[index] = i
        index += 1
        
    dframe = (dframe.rename(d1, axis = 'index'))
    return(dframe)
    

In [11]:
def trimName(teamName):    
    arryIndex = len(teamName)
    
    
    for a1 in range(arryIndex):
        #a1 is index for the string array teamName 
        strLen = len(teamName.iloc[a1])
        for b1 in teamName.iloc[a1][::-1]:
            if b1.isupper():
                strLen -= 1
            else:
                break
                
        teamName.iat[a1] = (teamName.iloc[a1][:strLen])
       
    return teamName

In [12]:
def setDataFrame(dframe, dataType, index, fixXa0):
    nameIndex = []
    dataMatrix = []
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    if(fixXa0):
        nameIndex = nameFix(nameIndex, frameArray[dataType]['\xa0'].get("Player")) 
        dframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    else:
        dframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(dataType == DEF):
            nameIndex = frameArray[dataType].get('Team')
            nameIndex = trimName(nameIndex)
        elif(dataType == KICKER):
            nameIndex = nameFix(nameIndex, frameArray[dataType].get('Player'))               
                               
        d1 = dict()
        index = 0

        for i in nameIndex:
            d1[index] = i
            index += 1
        
        dframe = (dframe.rename(d1, axis = 'index'))
        dframe = dframe.rename({'Pts*': 'Pts'}, axis='columns')
        
        
        
    return(dframe)



In [13]:
#MAIN STARTS HERE
frameArray = []
loadWebData(frameArray)

In [14]:
#Create multi-index from tuple
index = pd.MultiIndex.from_tuples(tupOffense)
frameQB = setDataFrame(frameArray[QB], QB, index, fixXa0 = True)
frameRB = setDataFrame(frameArray[RB], RB, index, fixXa0 = True)
frameWR = setDataFrame(frameArray[WR], WR, index, fixXa0 = True)
frameTE = setDataFrame(frameArray[TE], TE, index, fixXa0 = True)

index = pd.MultiIndex.from_tuples(tupDef)
frameDef = setDataFrame(frameArray[DEF], DEF, index, fixXa0 = False)

index = pd.MultiIndex.from_tuples(tupKicker)
frameKicker = setDataFrame(frameArray[KICKER], KICKER, index, fixXa0 = False)



In [15]:
columns = pd.MultiIndex.from_tuples(masterTup)
frameMaster = DataFrame(columns = columns)

In [16]:
frame = [frameMaster, frameKicker, frameQB]
frameMaster = pd.concat(frame)

\
                     Bye Game  Opp  PPR Position    Pts Team Week Year   
Greg Zuerlein        8.0  NaN  NaN  NaN      NaN  170.0  NaN  NaN  NaN   
Stephen Gostkowski   9.0  NaN  NaN  NaN      NaN  164.0  NaN  NaN  NaN   
Robbie Gould        11.0  NaN  NaN  NaN      NaN  153.0  NaN  NaN  NaN   
Matt Bryant          0.0  NaN  NaN  NaN      NaN  153.0  NaN  NaN  NaN   
Justin Tucker       10.0  NaN  NaN  NaN      NaN  151.0  NaN  NaN  NaN   
Harrison Butker     10.0  NaN  NaN  NaN      NaN  150.0  NaN  NaN  NaN   
Chris Boswell        9.0  NaN  NaN  NaN      NaN  150.0  NaN  NaN  NaN   
Wil Lutz             0.0  NaN  NaN  NaN      NaN  148.0  NaN  NaN  NaN   
Matt Prater          7.0  NaN  NaN  NaN      NaN  144.0  NaN  NaN  NaN   
Kai Forbath          9.0  NaN  NaN  NaN      NaN  142.0  NaN  NaN  NaN   
Ryan Succop          8.0  NaN  NaN  NaN      NaN  140.0  NaN  NaN  NaN   
Stephen Hauschka     6.0  NaN  NaN  NaN      NaN  130.0  NaN  NaN  NaN   
Jake Elliott        10.0  NaN  NaN  NaN      NaN  127.0  NaN  NaN  NaN   
Phil Dawson          8.0  NaN  NaN  NaN      NaN  127.0  NaN  NaN  NaN   
Graham Gano         11.0  NaN  NaN  NaN      NaN  121.0  NaN  NaN  NaN   
Adam Vinatieri      11.0  NaN  NaN  NaN      NaN  119.0  NaN  NaN  NaN   
Chandler Catanzaro  11.0  NaN  NaN  NaN      NaN  108.0  NaN  NaN  NaN   
Brandon McManus      0.0  NaN  NaN  NaN      NaN  105.0  NaN  NaN  NaN   
Blair Walsh          6.0  NaN  NaN  NaN      NaN  100.0  NaN  NaN  NaN   
Ka'imi Fairbairn     7.0  NaN  NaN  NaN      NaN   98.0  NaN  NaN  NaN   
Cody Parkey          1.0  NaN  NaN  NaN      NaN   91.0  NaN  NaN  NaN   
Giorgio Tavecchio   10.0  NaN  NaN  NaN      NaN   87.0  NaN  NaN  NaN   
Randy Bullock        6.0  NaN  NaN  NaN      NaN   87.0  NaN  NaN  NaN   
Josh Lambo           8.0  NaN  NaN  NaN      NaN   83.0  NaN  NaN  NaN   
Patrick Murray       1.0  NaN  NaN  NaN      NaN   82.0  NaN  NaN  NaN   
Aldrick Rosas        8.0  NaN  NaN  NaN      NaN   80.0  NaN  NaN  NaN   
Mason Crosby         8.0  NaN  NaN  NaN      NaN   80.0  NaN  NaN  NaN   
Dan Bailey           6.0  NaN  NaN  NaN      NaN   77.0  NaN  NaN  NaN   
Zane Gonzalez        9.0  NaN  NaN  NaN      NaN   74.0  NaN  NaN  NaN   
Dustin Hopkins       0.0  NaN  NaN  NaN      NaN   60.0  NaN  NaN  NaN   
...                  ...  ...  ...  ...      ...    ...  ...  ...  ...   
T.J. Yates           7.0  NaN  NaN  NaN      NaN   41.0  NaN  NaN  NaN   
Kevin Hogan          9.0  NaN  NaN  NaN      NaN   38.0  NaN  NaN  NaN   
Sam Bradford         9.0  NaN  NaN  NaN      NaN   32.0  NaN  NaN  NaN   
Bryce Petty         11.0  NaN  NaN  NaN      NaN   24.0  NaN  NaN  NaN   
David Fales          1.0  NaN  NaN  NaN      NaN   20.0  NaN  NaN  NaN   
Paxton Lynch         0.0  NaN  NaN  NaN      NaN   18.0  NaN  NaN  NaN   
E.J. Manuel         10.0  NaN  NaN  NaN      NaN   15.0  NaN  NaN  NaN   
Geno Smith           8.0  NaN  NaN  NaN      NaN   13.0  NaN  NaN  NaN   
Ryan Mallett        10.0  NaN  NaN  NaN      NaN   13.0  NaN  NaN  NaN   
Nathan Peterman      6.0  NaN  NaN  NaN      NaN   12.0  NaN  NaN  NaN   
Landry Jones         9.0  NaN  NaN  NaN      NaN   12.0  NaN  NaN  NaN   
Patrick Mahomes     10.0  NaN  NaN  NaN      NaN   10.0  NaN  NaN  NaN   
Nate Sudfeld        10.0  NaN  NaN  NaN      NaN    7.0  NaN  NaN  NaN   
Sean Mannion         8.0  NaN  NaN  NaN      NaN    5.0  NaN  NaN  NaN   
Matt Cassel          8.0  NaN  NaN  NaN      NaN    5.0  NaN  NaN  NaN   
Joe Webb             6.0  NaN  NaN  NaN      NaN    3.0  NaN  NaN  NaN   
Cody Kessler         9.0  NaN  NaN  NaN      NaN    2.0  NaN  NaN  NaN   
Scott Tolzien       11.0  NaN  NaN  NaN      NaN    1.0  NaN  NaN  NaN   
Cooper Rush          6.0  NaN  NaN  NaN      NaN    1.0  NaN  NaN  NaN   
A.J. McCarron        6.0  NaN  NaN  NaN      NaN    1.0  NaN  NaN  NaN   
Chad Henne           8.0  NaN  NaN  NaN      NaN    0.0  NaN  NaN  NaN   
Taylor Heinicke      7.0  NaN  NaN  NaN      NaN    0.0  NaN  NaN  NaN   
Au